<a href="https://colab.research.google.com/github/hphuc-bme138/Skin-disease-dectection-using-ResNet-152-and-web-app-with-Flask-ngrok/blob/main/Skin_Disease_Detection_using_ResNet_152_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Install the environment and required libraries**

In [ ]:
!pip install tensorflow==2.9.0
!pip install keras==2.9.0

In [ ]:
from numpy.random import seed
seed(101)

import pandas as pd
import numpy as np
import keras
#from keras import backend as K

import tensorflow
tensorflow.random.set_seed(101)
from tensorflow.keras.applications.resnet import ResNet152, preprocess_input
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

import os

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline
from keras.models import Sequential
from keras.layers import (BatchNormalization, Dense, Dropout, Flatten)
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy

# **Create the directory structure**

In [ ]:
# Create a new directory
base_dir_1 = '/content/drive/MyDrive/.../base_dir_1'
os.mkdir(base_dir_1)

# [now we create 4 folder inside 'base_dir']
# train_dir: av, ak, ez, ps
# val_dir: av, ak, ez, ps

# create a path to 'base_dir' to which I will join the names of the new folders
# train_dir
train_dir = os.path.join(base_dir_1, 'train_dir')
os.mkdir(train_dir)

# val_dir
val_dir = os.path.join(base_dir_1,'val_dir')
os.mkdir(val_dir)

# test_dir
test_dir = os.path.join(base_dir_1, 'test_dir')
os.mkdir(test_dir)

# Create the new folders inside train_dir
av = os.path.join(train_dir, 'av')
os.mkdir(av)
ak = os.path.join(train_dir, 'ak')
os.mkdir(ak)
ez = os.path.join(train_dir, 'ez')
os.mkdir(ez)
ps = os.path.join(train_dir, 'ps')
os.mkdir(ps)

# Create the new folders inside val_dir
av = os.path.join(val_dir, 'av')
os.mkdir(av)
ak = os.path.join(val_dir, 'ak')
os.mkdir(ak)
ez = os.path.join(val_dir, 'ez')
os.mkdir(ez)
ps = os.path.join(val_dir, 'ps')
os.mkdir(ps)

# Create the new folders inside test_dir
av = os.path.join(test_dir, 'av')
os.mkdir(av)
ak = os.path.join(test_dir, 'ak')
os.mkdir(ak)
ez = os.path.join(test_dir, 'ez')
os.mkdir(ez)
ps = os.path.join(test_dir, 'ps')
os.mkdir(ps)


Create a stratified train/test/val set

In [ ]:
df_data = pd.read_csv('/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/Skin lesion full new_metadata.csv')

df_data.head(20)

In [ ]:
df_data.shape

In [ ]:
y = df_data['dx']
_, df_test = train_test_split(df_data, test_size = 0.15, random_state=101, stratify=y)
df_test.shape


In [ ]:
df_test['dx'].value_counts()

Create a train set that excludes images that are in the val/test set

In [ ]:
def identify_test_rows(x):
  # create a list of all the lesion id in the val set
  test_list = list(df_test['image_id'])


  if str(x) in test_list:
    return 'test'
  else:
    return 'train'

In [ ]:
df_data['train or test'] = df_data['image_id']
df_data['train or test'] = df_data['train or test'].apply(identify_test_rows)

df_train = df_data[df_data['train or test'] == 'train']

In [ ]:
print(len(df_train))
print(len(df_test))

In [ ]:
df_train.shape
df_test.shape

In [ ]:
del df_train['train or test']

In [ ]:
z = df_train['dx']
_, df_val = train_test_split(df_train, test_size = 0.18, random_state=101, stratify=z)

df_val.shape

In [ ]:
def identify_val_rows(x):
  # create a list of all the lesion id in the val set
  val_list = list(df_val['image_id'])


  if str(x) in val_list:
    return 'val'
  else:
    return 'train'

In [ ]:
df_train['train or val'] = df_train['image_id']
df_train['train or val'] = df_train['train or val'].apply(identify_val_rows)

df_train_1 = df_train[df_train['train or val'] == 'train']

In [ ]:
del df_train_1['train or val']

In [ ]:
df_val['dx'].value_counts()

In [ ]:
df_train_1['dx'].value_counts()

In [ ]:
df_data['dx'].value_counts()

# **Transfer** the images into the folders

In [ ]:
df_data.set_index('image_id', inplace=True)

In [ ]:
folder = os.listdir('/content/drive/MyDrive/.../Skin-lesion-full-new')
for file in folder:
  print(file)


In [ ]:
# Get a list of train and val images
train_list = list(df_train_1['image_id'])
val_list = list(df_val['image_id'])
test_list = list(df_test['image_id'])


In [ ]:
print(train_list)

In [ ]:
# Transfer the train images

for image in train_list:
  fname = image + '.jpg'
  label = df_data.loc[image,'dx']

  if fname in folder:
    # source path to image
    src = os.path.join('/content/drive/MyDrive.../Skin-lesion-full-new', fname)
    # Destination path to image
    dst = os.path.join('/content/drive/MyDrive/.../train_dir', label, fname)
    # Copy the image from  source to de destination
    shutil.copyfile(src, dst)

In [ ]:
# Transfer the val images

for image in val_list:
  fname = image + '.jpg'
  label = df_data.loc[image,'dx']

  if fname in folder:
    # source path to image
    src = os.path.join('/content/drive/MyDrive/.../Skin-lesion-full-new', fname)
    # Destination path to image
    dst = os.path.join('/content/drive/MyDrive.../val_dir', label, fname)
    # Copy the image from  source to de destination
    shutil.copyfile(src, dst)


# Transfer the test image

for image in test_list:
  fname = image + '.jpg'
  label = df_data.loc[image,'dx']

  if fname in folder:
    # source path to image
    src = os.path.join('/content/drive/MyDrive/.../Skin-lesion-full-new', fname)
    # Destination path to image
    dst = os.path.join('/content/drive/MyDrive/.../test_dir', label, fname)
    # Copy the image from  source to de destination
    shutil.copyfile(src, dst)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/.../train_dir/ak')))
print(len(os.listdir('/content/drive/MyDrive/.../train_dir/av')))
print(len(os.listdir('/content/drive/MyDrive/.../train_dir/ez')))
print(len(os.listdir('/content/drive/MyDrive/.../train_dir/ps')))

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/.../val_dir/ak')))
print(len(os.listdir('/content/drive/MyDrive/.../val_dir/av')))
print(len(os.listdir('/content/drive/MyDrive/.../val_dir/ez')))
print(len(os.listdir('/content/drive/MyDrive/.../val_dir/ps')))

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/.../test_dir/ak')))
print(len(os.listdir('/content/drive/MyDrive/.../test_dir/av')))
print(len(os.listdir('/content/drive/MyDrive/.../test_dir/ez')))
print(len(os.listdir('/content/drive/MyDrive/.../test_dir/ps')))

# Augmentation for **classes**

In [ ]:

class_list = ['ak', 'av', 'ez', 'ps']

for item in class_list:

    # We are creating temporary directories here because we delete these directories later
    # create a base dir
    aug_dir = '/content/drive/MyDrive/.../aug_dir'
    os.mkdir(aug_dir)
    # create a dir within the base dir to store images of the same class
    img_dir = os.path.join('/content/drive/.../aug_dir', 'img_dir')
    os.mkdir(img_dir)

    # Choose a class
    img_class = item

    # list all images in that directory
    img_list = os.listdir('/content/drive/MyDrive/.../train_dir/' + img_class)

    # Copy images from the class train dir to the img_dir e.g. class 'mel'
    for fname in img_list:
            # source path to image
            src = os.path.join('/content/drive/MyDrive/.../train_dir/' + img_class, fname)
            # destination path to image
            dst = os.path.join('/content/drive/MyDrive/.../aug_dir/img_dir', fname)
            # copy the image from the source to the destination
            shutil.copyfile(src, dst)


    # point to a dir containing the images and not to the images themselves
    path = aug_dir
    save_path = '/content/drive/MyDrive/.../base_dir_3/train_dir/' + img_class

    # Create a data generator
    datagen = ImageDataGenerator(
        rotation_range=180,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True,
        vertical_flip=True,
        #brightness_range=(0.9,1.1),
        fill_mode='nearest')

    batch_size = 50

    aug_datagen = datagen.flow_from_directory(path,
                                           save_to_dir=save_path,
                                           save_format='jpg',
                                                    target_size=(224,224),
                                                    batch_size=batch_size)



    # Generate the augmented images and add them to the training folders

    ###########

    num_aug_images_wanted = 4000 # total number of images we want to have in each class

    ###########

    num_files = len(os.listdir('/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/aug_dir/img_dir'))
    num_batches = int(np.ceil((num_aug_images_wanted-num_files)/batch_size))

    # run the generator and create about 4000 augmented images
    for i in range(0,num_batches):

        imgs, labels = next(aug_datagen)

    # delete temporary directory with the raw image files
    shutil.rmtree('/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/aug_dir')

# **Some featured images after augmentation of datasets**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
import keras
from keras.preprocessing.image import ImageDataGenerator

# plots images with labels within jupyter notebook
def plots(ims, figsize=(12,6), rows=5, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0,2,3,1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')
plots(imgs, titles=None)

In [ ]:
print(len(os.listdir('/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/base_dir_3/train_dir/ak')))
print(len(os.listdir('/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/base_dir_3/val_dir/ak')))
print(len(os.listdir('/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/base_dir_3/test_dir/ak')))

In [ ]:
train_path = '/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/base_dir_3/train_dir'
valid_path = '/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/base_dir_3/val_dir'
test_path = '/content/drive/MyDrive/Dataset/skin-lesion-dataset-new/base_dir_3/test_dir'

IMAGE_SHAPE = (192, 256, 3)
data_gen_param = {
    "samplewise_center": True,
    "samplewise_std_normalization": True,
    "rotation_range": 90,
    "width_shift_range": 0.2,
    "height_shift_range": 0.2,
    "zoom_range": 0.2,
    "horizontal_flip": True,
    "vertical_flip": True,
    "rescale": 1.0 / 255
}
data_generator = ImageDataGenerator(**data_gen_param)

train_flow_param = {
    "directory": train_path,
    "batch_size": 64,
    "target_size": IMAGE_SHAPE[:2],
    "shuffle": True
}
train_flow = data_generator.flow_from_directory(**train_flow_param)

val_flow_param = {
    "directory": valid_path,
    "batch_size": 64,
    "target_size": IMAGE_SHAPE[:2],
    "shuffle": False
}
val_flow = data_generator.flow_from_directory(**val_flow_param)

test_flow_param = {
    "directory": test_path,
    "batch_size": 1,
    "target_size": IMAGE_SHAPE[:2],
    "shuffle": False
}
test_flow = data_generator.flow_from_directory(**test_flow_param)

In [ ]:
print(test_flow)

### **Built the model**

In [ ]:
from keras.models import Sequential
from keras.layers import (BatchNormalization, Dense, Dropout, Flatten)
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy

In [ ]:
pre_trained_model = ResNet152(input_shape= (192, 256, 3), include_top=False, weights = 'imagenet')

In [ ]:
for layer in pre_trained_model.layers:
    print(layer.name)
    layer.trainable = False

print(len(pre_trained_model.layers))

In [ ]:
last_layer = pre_trained_model.get_layer('conv5_block3_out')
print('last layer output shape:', last_layer.output_shape)
last_output = last_layer.output

In [ ]:
from tensorflow.keras import layers

In [ ]:
# Flatten the output layer to 1 dimension
x = layers.GlobalMaxPooling2D()(last_output)
# Add a fully connected layer with 512 hidden units and ReLU activation
x = layers.Dense(512, activation='relu')(x)
# Add a dropout rate of 0.5
x = layers.Dropout(0.5)(x)
# Add a final sigmoid layer for classification
x = layers.Dense(4, activation='softmax')(x)

In [ ]:


model = Model(pre_trained_model.input, x)

def top_2_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=2)

def top_3_acc(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred, k=3)

optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=[categorical_accuracy, top_2_acc, top_3_acc])

In [ ]:
model.summary()

In [ ]:
for layer in pre_trained_model.layers[:75]:
    layer.trainable = True

In [ ]:
optimizer = Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=True)

model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=[categorical_accuracy, top_2_acc, top_3_acc])

In [ ]:
model.summary()

In [ ]:
filepath = "/content/drive/MyDrive/.../ResNet152_28062023.h5"

checkpoint_param = {
    "filepath": filepath,
    "monitor": "val_categorical_accuracy",
    "verbose": 1,
    "save_best_only": True,
    "mode": "max"
}
checkpoint = ModelCheckpoint(**checkpoint_param)

lr_decay_params = {
    "monitor": "val_loss",
    "factor": 0.5,
    "patience": 3,
    "min_lr": 1e-5
}
lr_decay = ReduceLROnPlateau(**lr_decay_params)

early_stopping = EarlyStopping(monitor="val_loss", patience=10, verbose=1)

In [ ]:
num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 64
val_batch_size = 64


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)

In [ ]:
fit_params = {
    "generator": train_flow,
    "steps_per_epoch": train_steps,
    "epochs": 30,
    "verbose": 1,
    "validation_data": val_flow,
    "validation_steps": val_steps,
    "callbacks": [checkpoint, lr_decay, early_stopping]
}
print("Training the model...")

history = model.fit_generator(**fit_params)
print("Done!")

# **Confusion matrix**

In [ ]:
custom_object = {'top_2_acc':top_2_acc,
                  'top_3_acc':top_3_acc}

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
new_model_1 = tensorflow.keras.models.load_model('/content/drive/MyDrive/.../ResNet152.h5', custom_objects=custom_object)

In [ ]:
new_model_1.metrics_names

In [ ]:
test_labels = test_flow.classes

In [ ]:
test_labels

In [ ]:
test_labels.shape

In [ ]:
predictions = new_model_1.predict(test_flow, steps=len(df_test), verbose=1)

In [ ]:
classess = np.argmax(predictions)

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
cm_plot_labels = ['ak', 'av', 'ez', 'ps']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
y_pred = np.argmax(predictions, axis=1)
y_true = test_flow.classes

In [ ]:
from sklearn.metrics import classification_report
report = classification_report(y_true, y_pred, target_names = cm_plot_labels)

print(report)

# **ROC-AUC**

In [ ]:
import seaborn as sns
import pandas as pd
from sklearn.datasets import make_classification
from sklearn.preprocessing import label_binarize
from scipy import interp
from itertools import cycle
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc

In [ ]:
y_test = np.array(y_test)
n_classes = 4
x_test = X_test.astype('float32')/255

In [ ]:
pred_Y = new_model.predict(test_flow, steps=len(df_test), verbose=1)
# Plot linewidth.
lw = 2

In [ ]:
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], pred_Y[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
    # Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), pred_Y.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
# Plot of a ROC curve for a specific class
for i in range(n_classes):
    plt.figure()
    plt.plot(fpr[i], tpr[i], label='ROC curve (area = %0.2f)' % roc_auc[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

# Plot all ROC curves
fig = plt.figure(figsize=(12, 8))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f})'
               ''.format(roc_auc["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of class {0} (area = {1:0.2f})'
             ''.format(i, roc_auc[i]))


plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
sns.despine()
plt.show()

# **GRAD-CAM**

In [ ]:
label = ['ak', 'av', 'ez', 'ps']

In [ ]:
from collections import OrderedDict
from collections.abc import  Sequence

class _BaseWrapper(object):
    """
    Please modify forward() and backward() according to your task.
    """

    def __init__(self, model):
        super(_BaseWrapper, self).__init__()
        self.device = next(model.parameters()).device
        self.model = model
        self.handlers = []  # a set of hook function handlers

    def _encode_one_hot(self, ids):
        one_hot = torch.zeros_like(self.logits).to(self.device)
        one_hot.scatter_(1, ids, 1.0)
        return one_hot

    def forward(self, image):
        """
        Simple classification
        """
        self.model.zero_grad()
        self.logits = self.model(image)
        self.probs = F.softmax(self.logits, dim=1)
        return self.probs.sort(dim=1, descending=True)

    def backward(self, ids):
        """
        Class-specific backpropagation
        Either way works:
        1. self.logits.backward(gradient=one_hot, retain_graph=True)
        2. (self.logits * one_hot).sum().backward(retain_graph=True)
        """

        one_hot = self._encode_one_hot(ids)
        self.logits.backward(gradient=one_hot, retain_graph=True)

    def generate(self):
        raise NotImplementedError

    def remove_hook(self):
        """
        Remove all the forward/backward hook functions
        """
        for handle in self.handlers:
            handle.remove()


class GradCAM(_BaseWrapper):
    """
    "Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization"
    https://arxiv.org/pdf/1610.02391.pdf
    Look at Figure 2 on page 4
    """

    def __init__(self, model, candidate_layers=None):
        super(GradCAM, self).__init__(model)
        self.fmap_pool = OrderedDict()
        self.grad_pool = OrderedDict()
        self.candidate_layers = candidate_layers  # list

        def forward_hook(key):
            def forward_hook_(module, input, output):
                # Save featuremaps
                self.fmap_pool[key] = output.detach()

            return forward_hook_

        def backward_hook(key):
            def backward_hook_(module, grad_in, grad_out):
                # Save the gradients correspond to the featuremaps
                self.grad_pool[key] = grad_out[0].detach()

            return backward_hook_

        # If any candidates are not specified, the hook is registered to all the layers.
        for name, module in self.model.named_modules():
            if self.candidate_layers is None or name in self.candidate_layers:
                self.handlers.append(module.register_forward_hook(forward_hook(name)))
                self.handlers.append(module.register_backward_hook(backward_hook(name)))

    def _find(self, pool, target_layer):
        if target_layer in pool.keys():
            return pool[target_layer]
        else:
            raise ValueError("Invalid layer name: {}".format(target_layer))

    def _compute_grad_weights(self, grads):
        return F.adaptive_avg_pool2d(grads, 1)

    def forward(self, image):
        self.image_shape = image.shape[2:]
        return super(GradCAM, self).forward(image)

    def generate(self, target_layer):
        fmaps = self._find(self.fmap_pool, target_layer)
        grads = self._find(self.grad_pool, target_layer)
        weights = self._compute_grad_weights(grads)

        gcam = torch.mul(fmaps, weights).sum(dim=1, keepdim=True)
        gcam = F.relu(gcam)

        gcam = F.interpolate(
            gcam, self.image_shape, mode="bilinear", align_corners=False
        )

        B, C, H, W = gcam.shape
        gcam = gcam.view(B, -1)
        gcam -= gcam.min(dim=1, keepdim=True)[0]
        gcam /= gcam.max(dim=1, keepdim=True)[0]
        gcam = gcam.view(B, C, H, W)

        return gcam

In [ ]:
def demo2(image, label, model):
    """
    Generate Grad-CAM
    """
    # Model
    model = model
    model.to(device)
    model.eval()

    # The layers
    target_layers = ["conv2"]
    target_class = label

    # Images
    images = image.unsqueeze(0)
    gcam = GradCAM(model=model)
    probs, ids = gcam.forward(images)
    ids_ = torch.LongTensor([[target_class]] * len(images)).to(device)
    gcam.backward(ids=ids_)

    for target_layer in target_layers:
        print("Generating Grad-CAM @{}".format(target_layer))

        # Grad-CAM
        regions = gcam.generate(target_layer=target_layer)
        for j in range(len(images)):
            print(
                "\t#{}: {} ({:.5f})".format(
                    j, classes[target_class], float(probs[ids == target_class])
                )
            )

            gcam=regions[j, 0]
            plt.imshow(gcam.cpu())
            plt.show()